In [34]:
import sys
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import re
import subprocess

In [35]:
epochs = 80
trainable = True

In [36]:
from keras_contrib.layers import CRF


In [37]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [38]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [39]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [40]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [9]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [41]:
# conll
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


12152
2867
3005


In [42]:
print(train_sentences[100])

[['Erst', 'O'], ['in', 'O'], ['den', 'O'], ['bewegten', 'O'], ['politischen', 'O'], ['Zeiten', 'O'], ['der', 'O'], ['späten', 'O'], ['sechziger', 'O'], ['Jahre', 'O'], ['gab', 'O'], ['sie', 'O'], ['ihrem', 'O'], ['stürmischen', 'O'], ['Drängen', 'O'], ['nach', 'O'], ['.', 'O']]


In [43]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [44]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [45]:
# do lowercasing of words for fasttext embeddings. Not for others.
labelSet = set()
words = {}
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
#             words[token.lower()] = True
            words[re.sub('\d','#',token)] = True

In [46]:
print(labelSet)

{'I-MISC', 'I-LOC', 'B-LOC', 'B-ORG', 'O', 'B-MISC', 'B-PER', 'I-ORG', 'I-PER'}


In [47]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [48]:
print(label2Idx)

{'I-MISC': 0, 'B-PER': 6, 'I-LOC': 1, 'I-ORG': 7, 'B-LOC': 2, 'B-ORG': 3, 'I-PER': 8, 'B-MISC': 5, 'O': 4}


In [49]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [50]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/fastText/german_words_notfasttext.txt','wb')
f.write(string_words.encode())
f.close()

In [51]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'contains_digit': 6, 'other': 4, 'initialUpper': 3, 'PADDING_TOKEN': 7, 'allLower': 1, 'mainly_numeric': 5, 'numeric': 0, 'allUpper': 2}


In [64]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('german_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word.lower()]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [112]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_word2vecf.embeddings", encoding="utf-8")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0] in words:
#         print('yes')
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [ ]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_word2vec.embeddings.txt", 'rb')

for line in fEmbeddings:
    split = line.strip().split()
    word = split[0].decode()
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if word in words:
        
        for num in split[1:]:
            print(float(num))
#         vector = np.array([float(num.decode()) for num in split[1:]])
#         wordEmbeddings.append(vector)
#         word2Idx[split[0]] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [52]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_GloVe_w6.embeddings", encoding="utf-8")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0] in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [53]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings))
print(len(wordEmbeddings[2]))

31371
300


In [54]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'Q': 0, 'i': 1, '0': 3, 'Ä': 48, '?': 49, 'O': 4, 'u': 5, 'ô': 24, 'w': 6, 'x': 75, 'E': 51, 'a': 52, 'P': 53, 'o': 89, 'K': 8, 'ü': 2, 'k': 33, '(': 9, '§': 54, '&': 55, 'X': 56, 'B': 57, 'ò': 10, 'g': 11, 'á': 12, 'q': 58, 'H': 13, 'e': 59, '"': 60, '*': 34, ';': 82, 'n': 62, 'ç': 50, 'ó': 14, 'f': 15, '9': 63, 'Ö': 16, 'I': 64, 'R': 65, 'T': 17, '8': 18, 'z': 19, ')': 20, 'J': 32, 'F': 66, 'ñ': 67, '!': 68, 'ê': 69, 'N': 70, 'Ü': 22, 'y': 23, '7': 71, '/': 72, '6': 73, 'Z': 74, 'G': 25, 'é': 26, 'ß': 27, '2': 76, 'Y': 28, 'v': 94, '3': 29, "'": 77, '%': 61, 'p': 90, ',': 30, '-': 78, 'ì': 79, 'j': 31, 'D': 80, '5': 81, 'C': 92, 'W': 83, 'è': 35, 'S': 36, 'í': 37, ':': 84, 'U': 7, 'd': 38, '+': 39, 'b': 86, 'm': 40, '4': 85, 'V': 88, 'A': 87, 'L': 41, 'c': 91, 'r': 42, 'ö': 93, 'l': 21, 'h': 43, 'M': 44, '=': 95, '.': 96, '1': 45, 't': 46, 'ä': 97, 's': 47}


In [55]:
print(trainSentences[0]) 

[['Ereignis', ['E', 'r', 'e', 'i', 'g', 'n', 'i', 's'], 'O'], ['und', ['u', 'n', 'd'], 'O'], ['Erzählung', ['E', 'r', 'z', 'ä', 'h', 'l', 'u', 'n', 'g'], 'O'], ['oder', ['o', 'd', 'e', 'r'], 'O'], [':', [':'], 'O']]


In [56]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences, word2Idx,  label2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx, char2Idx))

In [57]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print((train_set[0][2]))

[[3728, 5, 3604, 55, 21], [3, 1, 3, 1, 4], array([[51, 42, 59,  1, 11, 62,  1, 47,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 5, 62, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [51, 42, 19, 97, 43, 21,  5, 62, 11,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [89, 38, 59, 42,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0

In [58]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [59]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [60]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 3136        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_11 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_14 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [63]:
for x in range(1):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
#             pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
            predLabels = np.concatenate(predLabels).ravel()
            correctLabels = np.concatenate(correctLabels).ravel()
            f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
#             print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
            print(f1_dev)
            if f1_dev > maxf1:
                model.save('german_ner_copy2_conll.h5')
                maxf1 = f1_dev
    #         predLabels, correctLabels = tag_dataset(dev_batch)        
        #     with open("dropout=0.75.txt",'w') as f:
    #         pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
        #     with open('dropout=0.75.txt') as f:
        #         x = str(epoch)+ " "+f1_dev
        #         f.write(x)
        #         f.write('\n')
    #         print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))


    

Epoch 0/80
100/79 [=====================================] - 15s 150ms/step
Epoch 1/80
100/79 [=====================================] - 15s 149ms/step
Epoch 2/80
100/79 [=====================================] - 16s 160ms/step
Epoch 3/80
100/79 [=====================================] - 16s 155ms/step
Epoch 4/80
100/79 [=====================================] - 15s 151ms/step
Epoch 5/80
100/79 [=====================================] - 15s 150ms/step
Epoch 6/80
100/79 [=====================================] - 16s 157ms/step
Epoch 7/80
100/79 [=====================================] - 16s 161ms/step
Epoch 8/80
100/79 [=====================================] - 15s 148ms/step
Epoch 9/80
100/79 [=====================================] - 15s 150ms/step
Epoch 10/80
100/79 [=====================================] - 16s 157ms/step
Epoch 11/80
100/79 [=====================================] - 16s 157ms/step
Epoch 12/80
100/79 [=====================================] - 16s 155ms/step
Epoch 13/80
100/79 [==

100/79 [=====================================] - 16s 161ms/step
Epoch 27/80
100/79 [=====================================] - 16s 163ms/step
Epoch 28/80
100/79 [=====================================] - 16s 159ms/step
Epoch 29/80
100/79 [=====================================] - 16s 158ms/step
Epoch 30/80
100/79 [=====================================] - 16s 156ms/step
Epoch 31/80
100/79 [=====================================] - 16s 162ms/step
Epoch 32/80
100/79 [=====================================] - 16s 160ms/step
Epoch 33/80
100/79 [=====================================] - 16s 161ms/step
Epoch 34/80
100/79 [=====================================] - 16s 162ms/step
Epoch 35/80
100/79 [=====================================] - 16s 161ms/step
Epoch 36/80
100/79 [=====================================] - 15s 155ms/step
Epoch 37/80
100/79 [=====================================] - 16s 158ms/step
Epoch 38/80
100/79 [=====================================] - 16s 158ms/step
Epoch 39/80
100/79 [====

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 51/80
2866/2867 [============================>.] - ETA: 0s0.544802574495478
Epoch 52/80
2866/2867 [============================>.] - ETA: 0s0.5580801519674148
Epoch 53/80
2866/2867 [============================>.] - ETA: 0s0.5472165916838897
Epoch 54/80
2866/2867 [============================>.] - ETA: 0s0.5576217683127926
Epoch 55/80
2866/2867 [============================>.] - ETA: 0s0.5563295595099241
Epoch 56/80
2866/2867 [============================>.] - ETA: 0s0.5523381974534386
Epoch 57/80
2866/2867 [============================>.] - ETA: 0s0.5508040901060848
Epoch 58/80
2866/2867 [============================>.] - ETA: 0s0.5504125118188454
Epoch 59/80
2866/2867 [============================>.] - ETA: 0s0.5552972144185844
Epoch 60/80
2866/2867 [============================>.] - ETA: 0s0.5524956938111968
Epoch 61/80
2866/2867 [============================>.] - ETA: 0s0.5553196861876429
Epoch 62/80
2866/2867 [============================>.] - ETA: 0s0.5458565991111656
Epoch

2866/2867 [============================>.] - ETA: 0s0.5585529485719164
Epoch 73/80
2866/2867 [============================>.] - ETA: 0s0.5643476127073257
Epoch 74/80
2866/2867 [============================>.] - ETA: 0s0.5612660985851609
Epoch 75/80
2866/2867 [============================>.] - ETA: 0s0.554221584434369
Epoch 76/80
2866/2867 [============================>.] - ETA: 0s0.5568618335510117
Epoch 77/80
2866/2867 [============================>.] - ETA: 0s0.5654331467869886
Epoch 78/80
2866/2867 [============================>.] - ETA: 0s0.5491440225792175
Epoch 79/80
2866/2867 [============================>.] - ETA: 0s0.5543613433907365


In [64]:
id2word = {v: k for k, v in word2Idx.items()}
print(len(correctLabels))        

model.load_weights('german_ner_copy2_conll.h5')
predLabels, correctLabels = tag_dataset(test_batch)        
# pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
# print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_glove6_conll'+str(x)+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]
        string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
#         string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
        string = string.encode()
        f.write(string)
        id+=1

# command = "perl nereval.perl"
# directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
command = "perl conlleval"
input_file= open(input_file_name)
output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_glove6_conll'+str(x)+'.txt'
output_file=open(output_file_name,'w')
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_conll)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

51444
3004/3005 [============================>.] - ETA: 0s******************************************************************************


In [30]:
import os
os.getcwd()

'/home/jindal/notebooks/jindal/NER'